In [1]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

In [2]:
import os

In [3]:
import tensorflow as tf
import numpy as np
import scipy
import time
from tensorflow.python.client import device_lib

In [4]:
device_lib.list_local_devices()

[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9533219572976025704, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 189792256
 locality {
   bus_id: 1
 }
 incarnation: 16551184755651625694
 physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:03:00.0"]

In [5]:
from model import Model_S2VT
from data_generator import Data_Generator
from inference_util import Inference

In [6]:
import inception_base

In [7]:
import configuration

In [8]:
data_config = configuration.DataConfig().config

In [9]:
data_gen = Data_Generator(data_config["processed_video_dir"],
                          data_config["caption_file"],
                          data_config["unique_frequency_cutoff"],
                          data_config["max_caption_length"])

In [10]:
#data_gen.build_dataset()
#data_gen.save(CAPTION_DATA_DIR)

In [11]:
data_gen.load_vocabulary(data_config["caption_data_dir"])
data_gen.load_dataset(data_config["caption_data_dir"])

In [ ]:
model_config = configuration.ModelConfig(data_gen).config

In [ ]:
model = Model_S2VT( num_frames = model_config["num_frames"],
                    image_width = model_config["image_width"],
                    image_height = model_config["image_height"],
                    image_channels = model_config["image_channels"],
                    num_caption_unroll = model_config["num_caption_unroll"],
                    num_last_layer_units = model_config["num_last_layer_units"],
                    image_embedding_size = model_config["image_embedding_size"],
                    word_embedding_size = model_config["word_embedding_size"],
                    hidden_size_lstm1 = model_config["hidden_size_lstm1"],
                    hidden_size_lstm2 = model_config["hidden_size_lstm2"],
                    vocab_size = model_config["vocab_size"],
                    initializer_scale = model_config["initializer_scale"],
                    learning_rate = model_config["learning_rate"])

In [ ]:
model.build()

In [ ]:
summary_op = tf.summary.merge(model.summaries)
train_writer = tf.summary.FileWriter(data_config["train_log_dir"],sess.graph)
val_writer = tf.summary.FileWriter(data_config["val_log_dir"],sess.graph)

In [ ]:
#model.initialize_inception_pretrained(sess,INCEPTION_PRETRAINED_CHECKPOINT)

In [ ]:
saver = tf.train.Saver(max_to_keep=20,keep_checkpoint_every_n_hours=1.0)

In [ ]:
model_path = tf.train.latest_checkpoint(data_config["checkpoint_dir"])

In [ ]:
sess = tf.Session()
#sess.run(tf.global_variables_initializer())

In [ ]:
if model_path != None:
    saver.restore(sess,model_path)

In [ ]:
BATCH_SIZE=64
data_gen.init_batch(BATCH_SIZE,"train")

In [ ]:
(data_gen.iter_per_epoch["train"]/2)

In [ ]:
iter_to_save

In [ ]:
infer_util = Inference(model,data_gen.word_to_idx,data_gen.idx_to_word)

In [ ]:
for epoch in range(NUM_EPOCHS):
    for i in range(data_gen.iter_per_epoch["train"]):
        start_time = time.time()
        dataset = data_gen.get_next_batch("train")
        data_gen_time = time.time() - start_time
        
        feed_dict={}
        feed_dict[model.caption_input] = dataset["indexed_caption"]
        feed_dict[model.caption_mask] = dataset["caption_mask"]
        feed_dict[model.rnn_input] = dataset["video"]
        
        if np.mod(i+1, 100) == 0:
            print("Writing Summary")
            summary,loss,global_step,_ = sess.run([summary_op,model.batch_loss,model.global_step,model.train_step],feed_dict=feed_dict)
            train_writer.add_summary(summary, global_step/100)
            
            time_global_step = tf.Summary()
            value = time_global_step.value.add()
            value.simple_value = (time.time() - start_time)
            value.tag = "global_step/time_global_step"
            train_writer.add_summary(time_global_step,global_step/100)
        else:
            loss,global_step,_ = sess.run([model.batch_loss,model.global_step,model.train_step],feed_dict=feed_dict)
        
        print("global_step = ",global_step,
              ", loss = ",loss,
              ', Elapsed time: %.2f' %(time.time() - start_time))        
        
        if np.mod(i+1,data_gen.iter_per_epoch["train"]/2) == 0:
            print("Saving the model ...")
            saver.save(sess, os.path.join(
                data_config["checkpoint_dir"], 'model'), global_step=int(global_step/data_gen.iter_per_epoch[MODE]))

In [ ]:
print("Saving the model ...")
saver.save(sess, os.path.join(
    data_config["checkpoint_dir"], 'model'), global_step=int(global_step/data_gen.iter_per_epoch[MODE]))

In [ ]:
data_gen.init_batch(BATCH_SIZE_TEST,"test")

In [ ]:
dataset = data_gen.get_next_batch("test")

In [ ]:
dataset["gen_caption"] = infer_util.generate_caption_batch(sess,dataset["video"],max_len=30)

In [ ]:
for i in range(10):
    print("File = %s \n  Real caption = %s \n   Gen Caption = %s" %(dataset['file'][i],dataset['caption'][i],dataset['gen_caption'][i]))


## TESTING

In [ ]:
ckpt_path = "/home/ozym4nd145/Coding/Notebook/SURA/code/s2vt_models/train"

In [ ]:
model_names = list(set([n.split(".")[0] for n in os.listdir(ckpt_path) if "model" in n]))

In [ ]:
model_names.sort(key= lambda x: int(x[6:]) )

In [ ]:
model_names

In [ ]:
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split('(\d+)', text) ]


In [ ]:
model_names.sort(key= natural_keys)

In [ ]:
model_names

In [ ]:
natural_keys(model_names[0])

In [ ]:
p = lambda x: int(x[5:])

In [ ]:
p(model_names[0])

In [12]:
len(data_gen.dataset["test"])

59800

In [13]:
data_gen.dataset["test"][0]

{'caption': 'a band performing in a small club',
 'caption_mask': array([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
 'file_name': 'video7960.mp4.npy',
 'indexed_caption': array([   1,    4,  125,  126,    7,    4,  131, 1233,    2,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32),
 'path': '../dataset/msrvtt_processed_video/test/video7960.mp4.npy',
 'processed_caption': '<bos> a band performing in a small club <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>',
 'real_caption': 'a band performing in a small club'}

In [15]:
data_gen.dataset["test"][0]

{'caption': 'a band performing in a small club',
 'caption_mask': array([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
 'file_name': 'video7960.mp4.npy',
 'indexed_caption': array([   1,    4,  125,  126,    7,    4,  131, 1233,    2,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32),
 'path': '../dataset/msrvtt_processed_video/test/video7960.mp4.npy',
 'processed_caption': '<bos> a band performing in a small club <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>',
 'real_caption': 'a band performing in a small club'}

In [41]:
videos = {i["file_name"]:i["path"] for i in data_gen.dataset["train"]}

In [42]:
len(videos)

6513

In [44]:
data_gen.init_batch(64,"train")

In [45]:
data_gen.iter_per_epoch

{'test': None, 'train': 2036, 'val': None}

In [46]:
2036*5

10180

In [24]:
for i in videos:
    print(i)

video8955.mp4.npy
video7377.mp4.npy
video9498.mp4.npy
video9821.mp4.npy
video9890.mp4.npy
video9493.mp4.npy
video9027.mp4.npy
video7864.mp4.npy
video8164.mp4.npy
video9975.mp4.npy
video9759.mp4.npy
video8770.mp4.npy
video9780.mp4.npy
video7736.mp4.npy
video9630.mp4.npy
video8677.mp4.npy
video8327.mp4.npy
video9945.mp4.npy
video8989.mp4.npy
video8608.mp4.npy
video7097.mp4.npy
video7503.mp4.npy
video7924.mp4.npy
video8762.mp4.npy
video7226.mp4.npy
video9831.mp4.npy
video9845.mp4.npy
video8341.mp4.npy
video9949.mp4.npy
video7719.mp4.npy
video7444.mp4.npy
video8889.mp4.npy
video7122.mp4.npy
video8263.mp4.npy
video7749.mp4.npy
video8072.mp4.npy
video9196.mp4.npy
video7291.mp4.npy
video9601.mp4.npy
video9751.mp4.npy
video9833.mp4.npy
video9128.mp4.npy
video7130.mp4.npy
video9997.mp4.npy
video9038.mp4.npy
video8209.mp4.npy
video7589.mp4.npy
video8391.mp4.npy
video9668.mp4.npy
video9252.mp4.npy
video7991.mp4.npy
video9748.mp4.npy
video8292.mp4.npy
video9486.mp4.npy
video7928.mp4.npy
video7894.

In [25]:
iter = 0

In [31]:
range(0,len(videos),50)

range(0, 2990, 50)

In [37]:
video_paths = {i["file_name"]:i["path"] for i in data_gen.dataset["test"]}
video_files = list(video_paths.keys())
        

In [38]:
video_files[0]

'video8955.mp4.npy'

In [39]:
len(video_files)

2990

In [40]:
2990/64

46.71875

In [ ]:
data_gen.dataset["train"]